# Análisis exploratorio (para el proyecto)

## Antes de partir...

### Requisitos

Usaremos los datos disponibles en [CONASET](https://mapas-conaset.opendata.arcgis.com/), particularmente [este de accidentes el año 2021](https://mapas-conaset.opendata.arcgis.com/datasets/4b636b2f27164b0ebbeca0ab9db4d08a_0/explore).

También usaremos la [cartografía del Censo 2017](https://github.com/PLUMAS-research/chile_census_2017_shapefiles). Para ello deben clonar ese repositorio y ejecutar el comando que aparece en su documentación: `find . -name "*.gz" | xargs gunzip` (en la carpeta de ese repositorio).

### Configuración

Además, utilizaremos un archivo de configuración `.env` que debe estar en la carpeta de estos notebooks, o bien, en su carpeta superior. Este archivo por el momento luce así:

```
AVES_ROOT=/home/egraells/resources/aves
CENSUS_GEO_ROOT=/mnt/d/centella/chile_census_2017_shapefiles/
```

La variable `AVES_ROOT` indica la carpeta donde está el repositorio `aves`; y la variable `CENSUS_GEO_ROOT`, donde se guardó el repositorio de cartografía del censo. Cargamos estas variables desde el archivo de configuración con la biblioteca `dotenv` (esto es necesario porque ustedes pueden ejecutar el notebook directamente sin tener que modificar rutas u otras cosas en el notebook).

In [1]:
import os
from dotenv import load_dotenv
from pathlib import Path

load_dotenv()

AVES_ROOT = Path(os.environ['AVES_ROOT'])
CENSUS_GEO_ROOT = Path(os.environ['CENSUS_GEO_ROOT'])

## Propuesta de proyecto

Esta podría ser una propuesta de proyecto inicial:

* **Situación**: La congestión y el tráfico en las ciudades es un fenómeno natural que emerge de las distintas actividades urbanas y su concentración espacial.
* **Complicación**: El transporte de personas también conlleva accidentes, con la correspondiente pérdida de vidas, problemas de salud y lesiones a las personas involucradas, daño a la propiedad pública y privada, y costo de tiempo a personas no involucradas.
* **Propuesta**: Caracterizar la accidentabilidad en Santiago para apoyar el diseño de políticas públicas que reduzcan la cantidad de accidentes.

Realizaremos un análisis exploratorio para ver su factibilidad e iterar  su definición.

## Bibliotecas necesarias

In [2]:
# para datos
import numpy as np
import pandas as pd
import geopandas as gpd
import requests

# visualización en Python
import seaborn as sns
import matplotlib.pyplot as plt

# AVES: Análisis, Visualización, Educación y Soporte
from aves.features.geo import clip_area_geodataframe, clip_point_geodataframe
from aves.visualization.figures import small_multiples_from_geodataframe
from aves.visualization.maps import choropleth_map, heat_map
from aves.config import setup_style
from aves.data.census.loading import read_census_map
from aves.features.utils import normalize_rows

setup_style()

## Construcción del dataset

Construir el dataset también involucra descargarlo y almacenarlo.

In [3]:
data_path = AVES_ROOT / "data" / "external" / "conaset"

if not data_path.exists():
    data_path.mkdir(parents=True)

In [4]:
file_path = data_path / "2019-rm-accidents.json"

if not file_path.exists():
    r = requests.get('https://opendata.arcgis.com/api/v3/datasets/4b636b2f27164b0ebbeca0ab9db4d08a_0/downloads/data?format=geojson&spatialRefId=4326&where=1%3D1')
    with open(file_path, 'wt') as f:
        f.write(r.text)

In [5]:
gdf = gpd.read_file(file_path, dtypes={'Fecha': 'str', 'Hora': 'str'}).to_crs('epsg:5361')

In [ ]:
gdf

In [ ]:
gdf.plot()

In [ ]:
gdf.columns

In [ ]:
gdf.dtypes

In [ ]:
gdf[['COMUNAREAL', 'Comuna_1']].value_counts()

In [ ]:
len(gdf[gdf['COMUNAREAL'] != gdf['Comuna_1']]) / len(gdf)

In [ ]:
comunas = read_census_map(
    "comuna", path=CENSUS_GEO_ROOT / "R13"
)

comunas.plot()

In [ ]:
limites_urbanos = read_census_map(
    "limite_urbano_censal", path=CENSUS_GEO_ROOT / "R13"
)

limites_urbanos.plot()

En http://bboxfinder.com podemos definir una _caja contenedora_ (_bounding box_) para área de análisis.

In [14]:
scl_bounds = [-70.88006218, -33.67612715, -70.43015094, -33.31069169]

In [ ]:
scl = clip_area_geodataframe(limites_urbanos, scl_bounds)
scl.plot()

## Limpiado, Filtrado y Pre-Procesamiento

In [16]:
# comunas_urbanas["NombreComuna"] = comunas_urbanas["NombreComuna"].replace(
#     {"Á": "A", "Ú": "U", "Ó": "O", "Í": "I", "É": "E"}, regex=True
# )


In [ ]:
ax = scl.plot(edgecolor="black", facecolor="none")
gdf[(gdf["COMUNAREAL"] != gdf["Comuna_1"]) & (gdf["Comuna_1"] == "SANTIAGO")].to_crs(scl.crs).plot(
    column="COMUNAREAL", ax=ax
)


In [ ]:
ax = scl.plot(edgecolor="black", facecolor="none")
gdf[(gdf["COMUNAREAL"] != gdf["Comuna_1"]) & (gdf["Comuna_1"] == "SANTIAGO")].to_crs(scl.crs).plot(
    column="Comuna_1", legend=False, ax=ax
)

Conclusión: Hay que usar "COMUNAREAL" si queremos estudiar comunas.

Cruzaremos los datos para quedarnos con los accidentes en el ámbito urbano.

In [ ]:
scl_accidents = clip_point_geodataframe(gdf.to_crs(scl.crs), scl_bounds)
scl_accidents.plot(alpha=0.01)

## ¿Qué contiene?

In [ ]:
scl_accidents.columns

In [ ]:
scl_accidents['Km_Vía_Fe'].value_counts().plot(kind='barh')

In [ ]:
scl_accidents['Condición'].value_counts().plot(kind='barh')

In [ ]:
scl_accidents['Tipo__CONA'].value_counts().plot(kind='barh')

In [ ]:
scl_accidents['Causa__CON'].value_counts().plot(kind='barh')

In [ ]:
scl_accidents['Fallecidos'].value_counts().plot(kind='barh')

In [ ]:
scl_accidents['Graves'].value_counts().plot(kind='barh')

In [ ]:
scl_accidents['Menos_Grav'].value_counts().plot(kind='barh')

In [ ]:
scl_accidents['Leves'].value_counts().plot(kind='barh')

In [ ]:
scl_accidents['Ilesos'].value_counts().plot(kind='barh')

In [30]:
scl_accidents["victimas"] = scl_accidents[
    ["Ilesos", "Graves", "Menos_Grav", "Fallecidos", "Leves"]
].sum(axis=1)

In [ ]:
scl_accidents["victimas"].value_counts(sort=False).sort_index().plot(
    kind="bar", logy=True
)

In [ ]:
scl_accidents['Estado_Atm'].value_counts().plot(kind='barh')

In [ ]:
scl_accidents['Estado_Cal'].value_counts().plot(kind='barh')

In [ ]:
scl_accidents['Ubicación'].value_counts().plot(kind='barh')

## ¿Cuándo?

In [ ]:
scl_accidents['Fecha']

In [ ]:
scl_accidents['Hora']

In [ ]:
scl_accidents.resample('1d', on='Fecha').size().plot()

In [ ]:
scl_accidents.resample('1W', on='Fecha').size().plot()

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(6, 12))

mean_accidents = (
    lambda name, x: x.resample("1d", on="Fecha")
    # .size()
    ["victimas"]
    .sum()
    .rolling(14, center=True)
    .mean()
    .rename(name)
    .to_frame()
    .apply(lambda x: (x - x.mean()) / x.std())
)

for ax, cat in zip(axes, ["COLISION", "CHOQUE", "ATROPELLO"]):
    # print(g)
    g = scl_accidents[scl_accidents["Tipo__CONA"] == cat]
    mean_accidents("all", scl_accidents).plot(ax=ax, color="grey", linewidth=0.5, legend=False)
    mean_accidents(cat, g).plot(ax=ax, linewidth=1.5, color="purple", legend=False)
    ax.set_title(cat)
    sns.despine(ax=ax, left=True, bottom=True)

fig.tight_layout()

## ¿Dónde?

In [ ]:
fig, ax = small_multiples_from_geodataframe(scl, 1, height=7)

scl.plot(ax=ax, facecolor="#efefef", edgecolor="none")
heat_map(ax, scl_accidents, weight="victimas", bandwidth=0.005, low_threshold=0.005)
scl.plot(ax=ax, facecolor="none", edgecolor="white")

In [ ]:
victimas_por_comuna = (
    scl_accidents.groupby(["COMUNAREAL", "Tipo__CONA"])["victimas"]
    .sum()
    .unstack(fill_value=0)
    .pipe(normalize_rows)
)
sns.clustermap(victimas_por_comuna, method="ward", annot=True, fmt='.2f')

In [ ]:
causas_por_comuna = (
    scl_accidents.groupby(["COMUNAREAL", "Causa__CON"])["victimas"]
    .sum()
    .unstack(fill_value=0)
    .pipe(normalize_rows)
)
sns.clustermap(causas_por_comuna, method="ward", annot=True, fmt='.2f')

## ¿Cómo?

In [ ]:
sns.clustermap(
    scl_accidents.groupby("Ubicación")["Tipo__CONA"]
    .value_counts()
    .unstack(fill_value=0)
    .pipe(normalize_rows), annot=True, fmt='.2f'
)

In [ ]:
sns.clustermap(
    scl_accidents.groupby("Tipo__CONA")["victimas"]
    .value_counts()
    .unstack(fill_value=0).T
    .pipe(np.sqrt)
    .pipe(normalize_rows).T
    , annot=True, fmt='.2f',
    col_cluster=False,
)

In [ ]:
sns.clustermap(
    scl_accidents.groupby("Tipo__CONA")["Causa__CON"]
    .value_counts()
    .unstack(fill_value=0)
    .pipe(normalize_rows), annot=True, fmt='.2f'
)

In [ ]:
sns.clustermap(
    scl_accidents.groupby("Ubicación")["Causa__CON"]
    .value_counts()
    .unstack(fill_value=0)
    .pipe(normalize_rows), annot=True, fmt='.2f'
)

In [47]:
scl_accidents['dia_de_semana'] = scl_accidents['Fecha'].dt.dayofweek
scl_accidents['fin_de_semana'] = scl_accidents['Fecha'].dt.dayofweek >= 5

In [ ]:
ax = scl_accidents.groupby("Causa__CON")['dia_de_semana'].value_counts().unstack(fill_value=0).T.pipe(normalize_rows).plot(kind='bar', stacked=True, linewidth=0, width=1.0, cmap='plasma')
ax.legend(bbox_to_anchor=(1.0, 0.0, 1.1, 1.0), loc='center left', reverse=True)#.pipe(lambda x: x[True] / x[False]).sort_values().plot(kind='barh')

# Entonces...

¿Es factible hacer algo con estos datos? Sí, aunque pareciera que se debe hacer algo a nivel general. La granularidad espacial es buena, pero temporalmente se pierde información o no hay variaciones grandes. Además hay pocos meta-datos de los accidentes relacionados con las condiciones en las que sucedieron, por ej., no se incluye el vehículo o las características de las personas.

Quizás lo que más llama la atención es la variación en la proporción de accidentes por día de semana. Ya que tenemos variabilidad espacial y en la proporción de accidentes por día de semana (o bien semana y fin de semana), podríamos iterar nuestro proyecto para que el objetivo sea aportar evidencia para:

1. Desarrollar estrategias de reducción de imprudencia de conductores. La tarea a resolver es **encontrar la relación** entre ubicación, accidentes e infraestructura del lugar. Posiblemente se requieran datos adicionales sobre el entorno construido;
2. Intensificar controles de alcoholemia los fines de semana. La tarea a resolver es **identificar** puntos críticos de accidentes en tipos de días específicos.
   
¡En estos casos, el proyecto parece factible!

¿Propones otros análisis futuros?